[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/feiglab/ficoll-generator/blob/main/BuildFicoll.ipynb)

# Building Ficoll Polymers

This notebook assembles Ficoll polymers of arbitrary size based on given input parameters

We begin by loading modules.

In [31]:
import os
import sys
import numpy as np
import math
import copy
import random

In [32]:
import subprocess

pkgs = [ i.split('==')[0].lower() for i in subprocess.getoutput('pip list --format=freeze').splitlines() ]
if not 'mdtraj' in pkgs:
    print('MDTraj is not available. Trying to install package.\n')
    !pip install mdtraj

In [33]:
import mdtraj as md

Let's begin by reading glycerol and sucrose molecules in a few different conformations.

The following cell downloads the PDB files from github if they are not available.

In [34]:
from pathlib import Path
if not Path("input/glycerol1.pdb").exists():
    print('downloading input structures')
    !mkdir -p input
    !wget -q https://raw.githubusercontent.com/feiglab/ficoll-generator/main/input/glycerol1.pdb -O input/glycerol1.pdb
    !wget -q https://raw.githubusercontent.com/feiglab/ficoll-generator/main/input/glycerol2.pdb -O input/glycerol2.pdb
    !wget -q https://raw.githubusercontent.com/feiglab/ficoll-generator/main/input/glycerol3.pdb -O input/glycerol3.pdb
    !wget -q https://raw.githubusercontent.com/feiglab/ficoll-generator/main/input/sucrose1.pdb -O input/sucrose1.pdb
    !wget -q https://raw.githubusercontent.com/feiglab/ficoll-generator/main/input/sucrose2.pdb -O input/sucrose2.pdb
    !wget -q https://raw.githubusercontent.com/feiglab/ficoll-generator/main/input/sucrose3.pdb -O input/sucrose3.pdb
    !wget -q https://raw.githubusercontent.com/feiglab/ficoll-generator/main/input/sucrose4.pdb -O input/sucrose4.pdb
        

In [35]:
class Molecule:
    def __init__(m,fname):
        m.xyz={}
        m.pdb=md.load_pdb(fname)
        m.mapatom={}
        for n in m.pdb.topology.atoms:
            m.mapatom[f'{n.residue.name}:{n.name}']=n.index
            
    def center(m):
        return md.compute_center_of_mass(m.pdb)[0]
        
    def getatom(m,key):
        return np.array(m.pdb.xyz[0][m.mapatom[key]])
        
gmol=[]
gmol+=[Molecule('input/glycerol1.pdb')]
gmol+=[Molecule('input/glycerol2.pdb')]
gmol+=[Molecule('input/glycerol3.pdb')]

smol=[]
smol+=[Molecule('input/sucrose1.pdb')]
smol+=[Molecule('input/sucrose2.pdb')]
smol+=[Molecule('input/sucrose3.pdb')]
smol+=[Molecule('input/sucrose4.pdb')]

In [36]:
def bond(mol,res,a1,a2):
    inx1=mol.pdb.topology.select(f"resname {res} and name {a1}")
    inx2=mol.pdb.topology.select(f"resname {res} and name {a2}")
    
    b=mol.pdb.xyz[0][inx1]-mol.pdb.xyz[0][inx2]

    return np.linalg.norm(b)

def angle(mol,res,a1,a2,a3):
    inx1=mol.pdb.topology.select(f"resname {res} and name {a1}")
    inx2=mol.pdb.topology.select(f"resname {res} and name {a2}")
    inx3=mol.pdb.topology.select(f"resname {res} and name {a3}")

    b1=mol.pdb.xyz[0][inx1]-mol.pdb.xyz[0][inx2]
    b2=mol.pdb.xyz[0][inx3]-mol.pdb.xyz[0][inx2]

    return math.acos(np.dot(b1[0],b2[0])/(np.linalg.norm(b1)*np.linalg.norm(b2)))*180.0/math.pi

def dihedral(mol,res,a1,a2,a3,a4):
    inx1=mol.pdb.topology.select(f"resname {res} and name {a1}")
    inx2=mol.pdb.topology.select(f"resname {res} and name {a2}")
    inx3=mol.pdb.topology.select(f"resname {res} and name {a3}")
    inx4=mol.pdb.topology.select(f"resname {res} and name {a4}")

    b1=mol.pdb.xyz[0][inx1]-mol.pdb.xyz[0][inx2]
    b2=mol.pdb.xyz[0][inx3]-mol.pdb.xyz[0][inx2]
    b3=mol.pdb.xyz[0][inx4]-mol.pdb.xyz[0][inx3]

    b0xb1 = np.cross(b1[0], b2[0])
    b1xb2 = np.cross(b3[0], b2[0])
    b0xb1_x_b1xb2 = np.cross(b0xb1, b1xb2)

    y = np.dot(b0xb1_x_b1xb2, b2[0])*(1.0/np.linalg.norm(b2))
    x = np.dot(b0xb1, b1xb2)

    return np.degrees(np.arctan2(y, x))

In [37]:
class GlycerolTopology:
    def __init__(g,mol):
        g.hlist=mol.pdb.topology.select("not type H")
        g.o1inx=mol.pdb.topology.select("name O1")
        g.c1inx=mol.pdb.topology.select("name C1")
        g.c2inx=mol.pdb.topology.select("name C2")
        g.c3inx=mol.pdb.topology.select("name C3")
        g.o3inx=mol.pdb.topology.select("name O3")
        g.o2inx=mol.pdb.topology.select("name O2")
        g.excllist=mol.pdb.topology.select("not name O1 and not name C1 and not type H")
        g.calcIC(mol)

    def calcIC(g,mol):
        g.bond={}
        g.bond['O1-C1']=bond(mol,"MGL","O1","C1")
        g.bond['C1-C2']=bond(mol,"MGL","C1","C2")
        g.bond['C2-C3']=bond(mol,"MGL","C2","C3")
        g.bond['C3-O3']=bond(mol,"MGL","C3","O3")
        g.bond['C2-O2']=bond(mol,"MGL","C2","O2")
        
        g.angle={}
        g.angle['O1-C1-C2']=angle(mol,"MGL","O1","C1","C2")
        g.angle['C1-C2-C3']=angle(mol,"MGL","C1","C2","C3")
        g.angle['C2-C3-O3']=angle(mol,"MGL","C2","C3","O3")
        g.angle['C1-C2-O2']=angle(mol,"MGL","C1","C2","O2") 
        
        g.dihedral={}
        g.dihedral['O1-C1-C2-C3']=dihedral(mol,"MGL","O1","C1","C2","C3")
        g.dihedral['C1-C2-C3-O3']=dihedral(mol,"MGL","C1","C2","C3","O3")
        g.dihedral['O1-C1-C2-O2']=dihedral(mol,"MGL","O1","C1","C2","O2")  
            

In [38]:
class SucroseTopology:
    def __init__(s,mol):
        s.calcIC(mol)
        
        s.hlist=mol.pdb.topology.select("not type H")
        
        s.o1inx={}
        s.c1inx={}
        s.c2inx={}
        s.c3inx={}
        s.c4inx={}
        
        s.excllist={}
        
        s.o1inx['f1']=mol.pdb.topology.select("resname BFRU and name O1")
        s.c1inx['f1']=mol.pdb.topology.select("resname BFRU and name C1")
        s.c2inx['f1']=mol.pdb.topology.select("resname BFRU and name C2")
        s.c3inx['f1']=mol.pdb.topology.select("resname BFRU and name C3")
        s.c4inx['f1']=mol.pdb.topology.select("resname BFRU and name C4")

        s.excllist['f1']=mol.pdb.topology.select(\
                f"not index {s.o1inx['f1'][0]} and not index {s.c1inx['f1'][0]} and \
                  not index {s.c2inx['f1'][0]} and not type H")
        
        s.o1inx['f6']=mol.pdb.topology.select("resname BFRU and name O6")
        s.c1inx['f6']=mol.pdb.topology.select("resname BFRU and name C6")
        s.c2inx['f6']=mol.pdb.topology.select("resname BFRU and name C5")
        s.c3inx['f6']=mol.pdb.topology.select("resname BFRU and name C4")
        s.c4inx['f6']=mol.pdb.topology.select("resname BFRU and name C3")

        s.excllist['f6']=mol.pdb.topology.select(\
                f"not index {s.o1inx['f6'][0]} and not index {s.c1inx['f6'][0]} and \
                  not index {s.c2inx['f6'][0]} and not type H")
        
        s.o1inx['g2']=mol.pdb.topology.select("resname AGLC and name O2")
        s.c1inx['g2']=mol.pdb.topology.select("resname AGLC and name C2")
        s.c2inx['g2']=mol.pdb.topology.select("resname AGLC and name C3")
        s.c3inx['g2']=mol.pdb.topology.select("resname AGLC and name C4")
        s.c4inx['g2']=mol.pdb.topology.select("resname AGLC and name C5")
        
        s.excllist['g2']=mol.pdb.topology.select(\
                f"not index {s.o1inx['g2'][0]} and not index {s.c1inx['g2'][0]} and \
                  not index {s.c2inx['g2'][0]} and not type H")

        s.o1inx['g6']=mol.pdb.topology.select("resname AGLC and name O6")
        s.c1inx['g6']=mol.pdb.topology.select("resname AGLC and name C6")
        s.c2inx['g6']=mol.pdb.topology.select("resname AGLC and name C5")
        s.c3inx['g6']=mol.pdb.topology.select("resname AGLC and name C4")
        s.c4inx['g6']=mol.pdb.topology.select("resname AGLC and name C3")
        
        s.excllist['g6']=mol.pdb.topology.select(\
                f"not index {s.o1inx['g6'][0]} and not index {s.c1inx['g6'][0]} and \
                  not index {s.c2inx['g6'][0]} and not type H")

        s.o1inx['g3']=mol.pdb.topology.select("resname AGLC and name O3")
        s.c1inx['g3']=mol.pdb.topology.select("resname AGLC and name C3")
        s.c2inx['g3']=mol.pdb.topology.select("resname AGLC and name C4")
        s.c3inx['g3']=mol.pdb.topology.select("resname AGLC and name C5")
        s.c4inx['g3']=mol.pdb.topology.select("resname AGLC and name C6")

        s.excllist['g3']=mol.pdb.topology.select(\
                f"not index {s.o1inx['g3'][0]} and not index {s.c1inx['g3'][0]} and \
                  not index {s.c2inx['g3'][0]} and not type H")
        
    def calcIC(s,mol):
        s.bond={}
        s.bond['f1']={}
        s.bond['f1']['O1-C1']=bond(mol,"BFRU","O1","C1")
        s.bond['f1']['C1-C2']=bond(mol,"BFRU","C1","C2")
        s.bond['f1']['C2-C3']=bond(mol,"BFRU","C2","C3")
        s.bond['f1']['C3-C4']=bond(mol,"BFRU","C3","C4")

        s.bond['f6']={}
        s.bond['f6']['O1-C1']=bond(mol,"BFRU","O6","C6")
        s.bond['f6']['C1-C2']=bond(mol,"BFRU","C6","C5")
        s.bond['f6']['C2-C3']=bond(mol,"BFRU","C5","C4")
        s.bond['f6']['C3-C4']=bond(mol,"BFRU","C4","C3")

        s.bond['g2']={}
        s.bond['g2']['O1-C1']=bond(mol,"AGLC","O2","C2")
        s.bond['g2']['C1-C2']=bond(mol,"AGLC","C2","C3")
        s.bond['g2']['C2-C3']=bond(mol,"AGLC","C3","C4")
        s.bond['g2']['C3-C4']=bond(mol,"AGLC","C4","C5")

        s.bond['g6']={}
        s.bond['g6']['O1-C1']=bond(mol,"AGLC","O6","C6")
        s.bond['g6']['C1-C2']=bond(mol,"AGLC","C6","C5")
        s.bond['g6']['C2-C3']=bond(mol,"AGLC","C5","C4")
        s.bond['g6']['C3-C4']=bond(mol,"AGLC","C4","C3")

        s.bond['g3']={}
        s.bond['g3']['O1-C1']=bond(mol,"AGLC","O3","C3")
        s.bond['g3']['C1-C2']=bond(mol,"AGLC","C3","C4")
        s.bond['g3']['C2-C3']=bond(mol,"AGLC","C4","C5")
        s.bond['g3']['C3-C4']=bond(mol,"AGLC","C5","C6")

        s.angle={}
        s.angle['f1']={}
        s.angle['f1']['O1-C1-C2']=angle(mol,"BFRU","O1","C1","C2")
        s.angle['f1']['C1-C2-C3']=angle(mol,"BFRU","C1","C2","C3")
        s.angle['f1']['C2-C3-C4']=angle(mol,"BFRU","C2","C3","C4")

        s.angle['f6']={}
        s.angle['f6']['O1-C1-C2']=angle(mol,"BFRU","O6","C6","C5")
        s.angle['f6']['C1-C2-C3']=angle(mol,"BFRU","C6","C5","C4")
        s.angle['f6']['C2-C3-C4']=angle(mol,"BFRU","C5","C4","C3")
    
        s.angle['g2']={}
        s.angle['g2']['O1-C1-C2']=angle(mol,"AGLC","O2","C2","C3")
        s.angle['g2']['C1-C2-C3']=angle(mol,"AGLC","C2","C3","C4")
        s.angle['g2']['C2-C3-C4']=angle(mol,"AGLC","C3","C4","C5")

        s.angle['g6']={}
        s.angle['g6']['O1-C1-C2']=angle(mol,"AGLC","O6","C6","C5")
        s.angle['g6']['C1-C2-C3']=angle(mol,"AGLC","C6","C5","C4")
        s.angle['g6']['C2-C3-C4']=angle(mol,"AGLC","C5","C4","C3")

        s.angle['g3']={}
        s.angle['g3']['O1-C1-C2']=angle(mol,"AGLC","O3","C3","C4")
        s.angle['g3']['C1-C2-C3']=angle(mol,"AGLC","C3","C4","C5")
        s.angle['g3']['C2-C3-C4']=angle(mol,"AGLC","C4","C5","C6")
                
        s.dihedral={}
        s.dihedral['f1']={}
        s.dihedral['f1']['O1-C1-C2-C3']=dihedral(mol,"BFRU","O1","C1","C2","C3")
        s.dihedral['f1']['C1-C2-C3-C4']=dihedral(mol,"BFRU","C1","C2","C3","C4")

        s.dihedral['f6']={}
        s.dihedral['f6']['O1-C1-C2-C3']=dihedral(mol,"BFRU","O6","C6","C5","C4")
        s.dihedral['f6']['C1-C2-C3-C4']=dihedral(mol,"BFRU","C6","C5","C4","C3")

        s.dihedral['g2']={}
        s.dihedral['g2']['O1-C1-C2-C3']=dihedral(mol,"AGLC","O2","C2","C3","C4")
        s.dihedral['g2']['C1-C2-C3-C4']=dihedral(mol,"AGLC","C2","C3","C4","C5")

        s.dihedral['g6']={}
        s.dihedral['g6']['O1-C1-C2-C3']=dihedral(mol,"AGLC","O6","C6","C5","C4")
        s.dihedral['g6']['C1-C2-C3-C4']=dihedral(mol,"AGLC","C6","C5","C4","C3")

        s.dihedral['g3']={}
        s.dihedral['g3']['O1-C1-C2-C3']=dihedral(mol,"AGLC","O3","C3","C4","C5")
        s.dihedral['g3']['C1-C2-C3-C4']=dihedral(mol,"AGLC","C3","C4","C5","C6")


In [39]:
gtop=[]
for i in range(len(gmol)):
    gtop+=[GlycerolTopology(gmol[i])]

stop=[]
for i in range(len(smol)):
    stop+=[SucroseTopology(smol[i])]

In [40]:
class GlycerolUnit:
    def __init__(g,inx,mol,top):
        g.index=inx
        g.resid=inx
        g.mol=copy.deepcopy(mol)
        g.top=top
        g.mapsite={'C1':'MGL:C1', 'O1':'MGL:O1', 'C3':'MGL:C3', 'O3':'MGL:O3'}
        g.connect={}
        g.connectsite={}

    def connectTo(g,site,sucrose,ssite):
        g.connect[site]=sucrose
        g.connectsite[site]=ssite
        
    def nconnect(g):
        return(len(g.connect.values()))
    
    def getatom(g,site,atom):
        return g.mol.getatom(g.mapsite[f'{atom}{site}'])

In [41]:
class SucroseUnit:
    def __init__(s,inx,mol,top):
        s.index=inx
        s.resid=inx
        s.mol=copy.deepcopy(mol)
        s.mapsite={'Cf1':'BFRU:C1', 'Of1':'BFRU:O1',\
                   'Cf6':'BFRU:C6', 'Of6':'BFRU:O6',\
                   'Cg2':'AGLC:C2', 'Og2':'AGLC:O2',\
                   'Cg6':'AGLC:C6', 'Og6':'AGLC:O6',\
                   'Cg3':'AGLC:C3', 'Og3':'AGLC:O3'}
        
        s.connect={}
        s.connectsite={}
        s.top=top
    
    def connectTo(s,site,glycerol,gsite):
        s.connect[site]=glycerol
        s.connectsite[site]=gsite
        
    def nconnect(s):
        return(len(s.connect.values()))
    
    def nlink(s):
        n=0
        for g in s.connect.values():
            if g.nconnect()>1: n+=1
        return n
    
    def nlone(s):
        return s.nconnect()-s.nlink()

    def getatom(s,site,atom):
        return s.mol.getatom(s.mapsite[f'{atom}{site}'])
    

In [42]:
class Ficoll:
    mw={}
    mw['sucrose'] = 342.301
    mw['glycerol'] = 92.0952
    mw['linkdel'] = (15.9994+1.008+1.008)
    
    seg='PROF'
    
    def __init__(f):
        f.sucrose=[]
        f.glycerol=[]
    
    def closest(f,m,list):        
        mind=999999.0
        
        for s in f.sucrose:
            d=minDist(s,m,s.top.hlist,list)
            if (d<mind):
                mind=d
        for g in f.glycerol:
            d=minDist(g,m,g.top.hlist,list)
            if (d<mind):
                mind=d
        return mind
    
    def alignSucroseToGlycerol(f,suc,ssite,igly,gsite):
        cg=f.glycerol[igly].getatom(gsite,'C')
        og=f.glycerol[igly].getatom(gsite,'O')
        
        
        spdb=suc.mol.pdb
        tpdb=copy.deepcopy(spdb)
        
        os=og
        tpdb.xyz[0][suc.top.o1inx[ssite]]=[os]      
        
        cs=buildFromIC(f.glycerol[igly].mol.center(),cg,os,0.143,112.0,160.0)
        tpdb.xyz[0][suc.top.c1inx[ssite]]=[cs]    
        
        fitinx=[suc.top.o1inx[ssite][0], suc.top.c1inx[ssite][0], suc.top.c2inx[ssite][0], \
                suc.top.c3inx[ssite][0], suc.top.c4inx[ssite][0]]
        
        done=False
        ncnt=1
        while not done and ncnt<10:
            rtor=random.random()*360.0-180.0
            c2s=buildFromIC(cg,os,cs,suc.top.bond[ssite]['C1-C2'],suc.top.angle[ssite]['O1-C1-C2'],rtor)
            tpdb.xyz[0][suc.top.c2inx[ssite]]=[c2s]      
            c3s=buildFromIC(os,cs,c2s,suc.top.bond[ssite]['C2-C3'],suc.top.angle[ssite]['C1-C2-C3'],suc.top.dihedral[ssite]['O1-C1-C2-C3'])
            tpdb.xyz[0][suc.top.c3inx[ssite]]=[c3s]
            c4s=buildFromIC(cs,c2s,c3s,suc.top.bond[ssite]['C3-C4'],suc.top.angle[ssite]['C2-C3-C4'],suc.top.dihedral[ssite]['C1-C2-C3-C4'])
            tpdb.xyz[0][suc.top.c4inx[ssite]]=[c4s]
            
            spdb.superpose(tpdb,atom_indices=fitinx)
            d=f.closest(suc,suc.top.excllist[ssite])
            if (d>0.2):
                done=True
            else:
                ncnt+=1
                
        return done
        
    def alignGlycerolToSucrose(f,glyc,isuc,ssite):
        gpdb=glyc.mol.pdb
        tpdb=copy.deepcopy(gpdb)

        csuc=f.sucrose[isuc].getatom(ssite,'C')
        osuc=f.sucrose[isuc].getatom(ssite,'O')
        
        og=osuc
        tpdb.xyz[0][glyc.top.o1inx]=[og]
        
        cg=buildFromIC(f.sucrose[isuc].mol.center(),csuc,og,0.143,112.0,160.0)
        tpdb.xyz[0][glyc.top.c1inx]=[cg]    

        fitinx=[glyc.top.o1inx[0], glyc.top.c1inx[0], glyc.top.c2inx[0], glyc.top.c3inx[0], glyc.top.o2inx[0], glyc.top.o3inx[0]]
                       
        done=False
        ncnt=1
        while not done and ncnt<10:
            rtor=random.random()*360.0-180.0
            c2g=buildFromIC(csuc,og,cg,glyc.top.bond['C1-C2'],glyc.top.angle['O1-C1-C2'],rtor)
            tpdb.xyz[0][glyc.top.c2inx]=[c2g]   
            c3g=buildFromIC(og,cg,c2g,glyc.top.bond['C2-C3'],glyc.top.angle['C1-C2-C3'],glyc.top.dihedral['O1-C1-C2-C3'])
            tpdb.xyz[0][glyc.top.c3inx]=[c3g]
            o3g=buildFromIC(cg,c2g,c3g,glyc.top.bond['C3-O3'],glyc.top.angle['C2-C3-O3'],glyc.top.dihedral['C1-C2-C3-O3'])
            tpdb.xyz[0][glyc.top.o3inx]=[o3g]
            o2g=buildFromIC(og,cg,c2g,glyc.top.bond['C2-O2'],glyc.top.angle['C1-C2-O2'],glyc.top.dihedral['O1-C1-C2-O2'])
            tpdb.xyz[0][glyc.top.o2inx]=[o2g]
            
            gpdb.superpose(tpdb,atom_indices=fitinx)
            d=f.closest(glyc,glyc.top.excllist)
            if (d>0.2):
                done=True
            else:
                ncnt+=1
                
        return done
         
    def addSucrose(f,mol,top,minx,igly=-1,site=''):
        isuc=len(f.sucrose)
        s=SucroseUnit(isuc+1,mol[minx],top[minx])
        
        add=False
        if (igly>=0):
            ok=f.alignSucroseToGlycerol(s,site,igly,'3')
            if (ok):
                f.glycerol[igly].connectTo('3',s,site)
                s.connectTo(site,f.glycerol[igly],'3')
                add=True
        else:
            add=True
            
        if (add):
            f.sucrose+=[s]    
            return isuc
        else:
            return -1
    
    def addGlycerol(f,mol,top,minx,isuc=-1,site=''):
        igly=len(f.glycerol)
        g=GlycerolUnit(igly+1,mol[minx],top[minx])
        
        add=False
        if (isuc>=0):
            ok=f.alignGlycerolToSucrose(g,isuc,site)
            if (ok):
                f.sucrose[isuc].connectTo(site,g,'1')
                g.connectTo('1',f.sucrose[isuc],site)
                add=True
        else: 
            add=True
            
        if (add):
            f.glycerol+=[g]
            return igly
        else:            
            return -1
        
    def molecularweight(f):
        totmw=len(f.sucrose)*f.mw['sucrose']+len(f.glycerol)*f.mw['glycerol']
        for g in f.glycerol:
            totmw-=f.mw['linkdel']*g.nconnect()
        return totmw
    
    def nglycerol(f):
        return len(f.glycerol)

    def nsucrose(f):
        return len(f.sucrose)
    
    def glycerolPerSucrose(f):
        return f.nglycerol()/f.nsucrose()

    def glycerolLinkFraction(f):
        sum=0.0
        for g in f.glycerol:
            sum+=(g.nconnect()-1)
        return sum/f.nglycerol()
    
    def generate(f):
        ires=1
        for s in f.sucrose:
            s.resid=ires
            ires+=2
        for g in f.glycerol:
            g.resid=ires
            ires+=1
        
    def write(f,fname=''):
        f.generate()
        
        if fname:
            fout=open(fname,"w")
        else:
            fout=sys.stdout
            
        iatom=1
        for s in f.sucrose:
            p=s.mol.pdb
            t=p.topology
            for a in t.residue(0).atoms:
                if ('f1' in s.connect and (a.name == 'HO1')): 
                    continue
                elif ('f6' in s.connect and (a.name == 'HO6')):
                    continue
                printPDB(fout,a,iatom,s.resid,p.xyz[0][a.index],f.seg)
                iatom+=1
            
            for a in t.residue(1).atoms:
                if ('g2' in s.connect and (a.name == 'HO2')): 
                    continue
                elif ('g3' in s.connect and (a.name == 'HO3')):
                    continue
                elif ('g6' in s.connect and (a.name == 'HO6')):
                    continue
                printPDB(fout,a,iatom,s.resid+1,p.xyz[0][a.index],f.seg)
                iatom+=1
            
        for g in f.glycerol:
            p=g.mol.pdb
            t=p.topology
            for a in t.atoms:
                if ('1' in g.connect and (a.name == 'O1' or a.name == 'HO1')): 
                    continue
                elif ('3' in g.connect and (a.name == 'O3' or a.name == 'HO3')):
                    continue
            
                printPDB(fout,a,iatom,g.resid,p.xyz[0][a.index],f.seg)                    
                iatom+=1
        
        fout.write("TER\n")
        fout.write("END\n")
        if fname:
            fout.close()
        
    def patches(f,fname=''):
        f.generate()
        
        if fname:
            fout=open(fname,"w")
        else:
            fout=sys.stdout
        
        for s in f.sucrose:
            fout.write("SUCR:%s.%d:%s.%d\n" % (f.seg,s.resid+1,f.seg,s.resid))
            if 'f1' in s.connect:
                fout.write("FG1%s:%s.%d:%s.%d\n" % (s.connectsite['f1'],f.seg,s.resid,f.seg,s.connect['f1'].resid))
            if 'f6' in s.connect:
                fout.write("FG6%s:%s.%d:%s.%d\n" % (s.connectsite['f6'],f.seg,s.resid,f.seg,s.connect['f6'].resid))
            if 'g2' in s.connect:
                fout.write("GG2%s:%s.%d:%s.%d\n" % (s.connectsite['g2'],f.seg,s.resid+1,f.seg,s.connect['g2'].resid))
            if 'g3' in s.connect:
                fout.write("GG3%s:%s.%d:%s.%d\n" % (s.connectsite['g3'],f.seg,s.resid+1,f.seg,s.connect['g3'].resid))
            if 'g6' in s.connect:
                fout.write("GG6%s:%s.%d:%s.%d\n" % (s.connectsite['g6'],f.seg,s.resid+1,f.seg,s.connect['g6'].resid))
 
        if fname:
            fout.close()

In [43]:
def buildFromIC(a,b,c,r,theta,phi):   
    sinTheta=math.sin(theta*math.pi/180.0)
    cosTheta=math.cos(theta*math.pi/180.0)
    sinPhi=math.sin(phi*math.pi/180.0)
    cosPhi=math.cos(phi*math.pi/180.0)

    x=r*cosTheta
    y=r*cosPhi*sinTheta
    z=r*sinPhi*sinTheta
    
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    
    ab=b-a
    bc=c-b
    bc=bc/np.linalg.norm(bc)
    
    n=np.cross(ab,bc)
    n=n/np.linalg.norm(n)
    ncbc=np.cross(n,bc)
    
    m=np.array([[bc[0],ncbc[0],n[0]], [bc[1],ncbc[1],n[1]], [bc[2],ncbc[2],n[2]]])
    d=np.matmul(m,np.array([-x,y,z]))+c
    return(d.tolist())
    
def minDist(m1,m2,hlist,list):
    mind=999999.0
    for h in hlist:
        xyz1=m1.mol.pdb.xyz[0][h]
        for l in list:
            xyz2=m2.mol.pdb.xyz[0][l]
            dx=xyz2-xyz1
            #numpy.sqrt(numpy.sum((a - b) ** 2, axis=0))
            #d=np.linalg.norm(dx)
            d=np.sum(dx**2)
            if (d<mind):
                mind=d
    return np.sqrt(mind)

def printPDB(fout,a,natom,nres,xyz,seg):
    fout.write("ATOM%7d %-4s %-4s%1s%-6s  %8.3f%8.3f%8.3f %5.2f%6.2f      %-4s\n" % \
              (natom,a.name,a.residue.name," ",nres,xyz[0]*10.0,xyz[1]*10.0,xyz[2]*10.0,1.0,0.0,seg))

Example for how to assemble a molecule manually:

In [44]:
f=Ficoll()

is1=f.addSucrose(smol,stop,0)
ig1=f.addGlycerol(gmol,gtop,0,is1,'f6')
ig2=f.addGlycerol(gmol,gtop,1,is1,'f1')
ig3=f.addGlycerol(gmol,gtop,0,is1,'g2')

is2=f.addSucrose(smol,stop,0,ig1,'f1')
is3=f.addSucrose(smol,stop,0,ig2,'g2')
is4=f.addSucrose(smol,stop,0,ig3,'f1')

ig4=f.addGlycerol(gmol,gtop,2,is4,'g2')

print(f.molecularweight())
print(f.glycerolPerSucrose())

print('PDB file:')
f.write()
#f.write('ficoll-example.pdb')

print('\npatches:')
f.patches()
#f.patches('ficoll-example.patches')

1611.477
1.0
PDB file:
ATOM      1 O5   BFRU 1         -0.066  -1.940  -0.585  1.00  0.00      PROF
ATOM      2 C2   BFRU 1          0.166  -1.600   0.792  1.00  0.00      PROF
ATOM      3 C5   BFRU 1         -1.449  -2.294  -0.761  1.00  0.00      PROF
ATOM      4 H5   BFRU 1         -1.530  -3.391  -0.571  1.00  0.00      PROF
ATOM      5 C6   BFRU 1         -1.887  -1.977  -2.169  1.00  0.00      PROF
ATOM      6 H61  BFRU 1         -2.961  -2.243  -2.298  1.00  0.00      PROF
ATOM      7 H62  BFRU 1         -1.775  -0.887  -2.366  1.00  0.00      PROF
ATOM      8 O6   BFRU 1         -1.114  -2.701  -3.126  1.00  0.00      PROF
ATOM      9 C1   BFRU 1          1.204  -2.556   1.371  1.00  0.00      PROF
ATOM     10 H11  BFRU 1          2.152  -2.441   0.800  1.00  0.00      PROF
ATOM     11 H12  BFRU 1          1.399  -2.270   2.429  1.00  0.00      PROF
ATOM     12 O1   BFRU 1          0.808  -3.936   1.333  1.00  0.00      PROF
ATOM     13 C3   BFRU 1         -1.218  -1.670   1.49

Automatic assembly for generating larger Ficoll molecules

In [45]:
avgGlycerolPerSucrose=2.92    # from NMR data
probGlycerolLink=0.8          # probability of glycerol forming diether linkage

targetMW = 10000              # in Dalton


prob={}                       # probabilities below should add to zero
prob['f1']=0.3                # probability of glycerol linkage at fructose C1
prob['f6']=0.2                # probability of glycerol linkage at fructose C6
prob['g2']=0.25               # probability of glycerol linkage at glucose C2
prob['g6']=0.3                # probability of glycerol linkage at glucose C6
prob['g3']=0.05               # probability of glycerol linkage at glucose C3

def randomSite(p):
    v=random.random()
    sum=0.0
    for s in p:
        sum+=p[s]
        if (v<sum):
            return s
    return ''

In [46]:
f=Ficoll()

isuc=f.addSucrose(smol,stop,random.randrange(len(smol)))
islist=[isuc]
while(f.molecularweight()<targetMW and len(islist)>0):
    iglist=[]
    random.shuffle(islist)
    for s in islist:
        cnt=0
        while (f.glycerolPerSucrose()<avgGlycerolPerSucrose and cnt<10 and f.molecularweight()<targetMW):
            rsite=randomSite(prob)
                #print(rsite)
            if rsite in f.sucrose[s].connect:
                cnt+=1
                continue
            igly=f.addGlycerol(gmol,gtop,random.randrange(len(gmol)),s,rsite)
            if (igly>=0): iglist+=[igly]
        
    print("added %3d glycerols : mw %10.2lf : %5.2lf glyc/suc : %5.2lf links/glyc" %\
          (len(iglist), f.molecularweight(), f.glycerolPerSucrose(), f.glycerolLinkFraction()))

    islist=[]
    random.shuffle(iglist)
    for g in iglist:
        if (random.random()<probGlycerolLink*1.20 and f.molecularweight()<targetMW):
            rsite=randomSite(prob)
            isuc=f.addSucrose(smol,stop,random.randrange(len(smol)),g,rsite)
            if (isuc>=0): islist+=[isuc]
            
    if (len(islist)==0 and f.molecularweight()<targetMW):
        #print(f'add at least one')
        g=iglist[random.randrange(len(iglist))]
        rsite=randomSite(prob)
        isuc=f.addSucrose(smol[random.randrange(len(smol))],g,rsite)
        if (isuc>=0): islist+=[isuc]

    print("added %3d sucroses  : mw %10.2lf : %5.2lf glyc/suc : %5.2lf links/glyc" %\
          (len(islist), f.molecularweight(), f.glycerolPerSucrose(), f.glycerolLinkFraction()))


added   3 glycerols : mw     564.54 :  3.00 glyc/suc :  0.00 links/glyc
added   3 sucroses  : mw    1537.40 :  0.75 glyc/suc :  1.00 links/glyc
added   9 glycerols : mw    2204.12 :  3.00 glyc/suc :  0.25 links/glyc
added   8 sucroses  : mw    4798.40 :  1.00 glyc/suc :  0.92 links/glyc
added  23 glycerols : mw    6502.24 :  2.92 glyc/suc :  0.31 links/glyc
added  11 sucroses  : mw   10069.38 :  1.52 glyc/suc :  0.63 links/glyc


Saving molecule and writing out patches

In [47]:
f.write('ficoll.pdb')
f.patches('ficoll.patches')

On Google Colab, use File explorer to view and download generated files.

In [48]:
import subprocess

pkgs = [ i.split('==')[0].lower() for i in subprocess.getoutput('pip list --format=freeze').splitlines() ]
if not 'nglview' in pkgs:
    print('NGLview is not available. Trying to install package.\n')
    !pip install nglview

In [49]:
import nglview as nv

if 'google.colab' in sys.modules:
    from google.colab import output
    output.enable_custom_widget_manager()

viewpdb=md.load_pdb("ficoll.pdb")

view = nv.NGLWidget(nv.MDTrajTrajectory(viewpdb))
view.camera='orthographic'

view

NGLWidget()